In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import datetime as datetime
from tqdm import tqdm
import networkx as nx
import random

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from src.utils.CreateFeatures import CreateFeatures
from src.pygcn.SiameseGNN import SiameseGNN
from src.pygcn.GraphSAGE import SiameseGNN_GraphSAGE
from src.pygcn.graph_isomorphism import SiameseGNN_GIN
from torch_geometric.data import DataLoader
from sklearn.metrics import f1_score, fbeta_score
from src.utils.functions import dist_labels_to_changepoint_labels, dist_labels_to_changepoint_labels_adjusted
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch
import torch_geometric.data as data
from src.synthetic_experiments.sample import sample_pairs
from src.utils.misc import collate
import itertools

## All Events

In [2]:
years = range(1962,2019)

In [3]:
all_nodes = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA',
       'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM',
       'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI',
       'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR',
       'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'LTU',
       'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT',
       'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR',
       'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS',
       'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR',
       'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM',
       'SYC', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON',
       'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA',
       'UZB', 'VCT', 'VEN', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB',
       'ZWE']

In [4]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [5]:
crisis_years = [1962, 1967, 1973, 1978, 1981, 1989, 1993, 1996, 2002, 2007, 2012, 2014, 2016]
phases = []
p = -1
for i in range(1962,2019):
    if i in crisis_years:
        p += 1
    phases.append(p)

In [6]:
# labels = dist_labels_to_changepoint_labels(phases)
# graph_pairs = sample_pairs(all_graphs,labels)

In [7]:
# with open('graph_pairs.pkl', 'wb') as f:
#     pkl.dump(graph_pairs, f)

In [8]:
with open('graph_pairs.pkl', 'rb') as f:
    graph_pairs = pkl.load(f)

In [9]:
with open('test_data.pkl', 'rb') as f:
    test_data = pkl.load(f)

with open('val_data.pkl', 'rb') as f:
    val_data = pkl.load(f)

with open('train_data.pkl', 'rb') as f:
    train_data = pkl.load(f)

In [10]:
with open('test_data_indices.pkl', 'rb') as f:
    test_indices = pkl.load(f)

with open('val_data_indices.pkl', 'rb') as f:
    val_indices = pkl.load(f)

with open('train_data_indices.pkl', 'rb') as f:
    train_indices = pkl.load(f)

In [11]:
# train_indices, test_indices = train_test_split(np.arange(len(graph_pairs)), test_size=0.40, random_state=42)
# test_indices, val_indices = train_test_split(train_indices, test_size=0.5, random_state=42)

In [12]:
graph_pairs_train = [graph_pairs[i] for i in train_indices]
graph_pairs_test = [graph_pairs[i] for i in test_indices]
graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [13]:
import random
positive_samples = [item for item in graph_pairs_train if item[2] == 1]
negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = graph_pairs_train

random.shuffle(balanced_data)

In [14]:
# with open('train_data.pkl', 'wb') as f:
#     pkl.dump(balanced_data, f)

# with open('val_data.pkl', 'wb') as f:
#     pkl.dump(graph_pairs_val, f)
    
# with open('test_data.pkl', 'wb') as f:
#     pkl.dump(graph_pairs_test, f)

In [15]:
# with open('train_data_indices.pkl', 'wb') as f:
#     pkl.dump(train_indices, f)

# with open('val_data_indices.pkl', 'wb') as f:
#     pkl.dump(val_indices, f)
    
# with open('test_data_indices.pkl', 'wb') as f:
#     pkl.dump(test_indices, f)

In [17]:
def run_model(model, training_data_pairs, validation_data_pairs):
    torch.manual_seed(42)
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0001)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    criterion = nn.BCELoss()  # Changed to BCEWithLogitsLoss for numerical stability

    for epoch in tqdm(range(30)):
        model.train()
        train_losses = []
        for (graph1, graph2, labels) in training_data_pairs:
            optimizer.zero_grad()
            out = model(graph1, graph2)
    
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_losses = []

            val_pred = []
            val_truth = []

            correct = 0
            total = 0
            for (graph1, graph2, labels) in validation_data_pairs:
                out = model(graph1, graph2)

                val_loss = criterion(out, labels)
                val_losses.append(val_loss.item())

                predictions = torch.round(out)

                val_pred.extend(predictions.cpu().numpy())
                val_truth.extend(labels.cpu().numpy())

                correct += (predictions == labels).sum().item()
                total += labels.size(0)

            val_loss = sum(val_losses) / len(val_losses)
            val_accuracy = correct / total

        val_f1 = f1_score(val_truth, val_pred)
        val_f2 = fbeta_score(y_true=val_truth, y_pred=val_pred, beta=2)
        val_f05 = fbeta_score(y_true=val_truth, y_pred=val_pred, beta=1 / 2)
        print(f'Epoch: {epoch+1}, Training Loss: {sum(train_losses)/len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}, Validation F2 Score: {val_f2}, Validation F0.5 Score: {val_f05}')
    return val_accuracy, val_f1, val_loss, val_f2, val_f05

## sGNN with GCN Encoder and 3 Features

In [29]:
train_data

[[Data(x=[199, 12], edge_index=[2, 6037], edge_attr=[6037], y=[199]),
  Data(x=[199, 12], edge_index=[2, 11125], edge_attr=[11125], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 8656], edge_attr=[8656], y=[199]),
  Data(x=[199, 12], edge_index=[2, 15242], edge_attr=[15242], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 23901], edge_attr=[23901], y=[199]),
  Data(x=[199, 12], edge_index=[2, 8107], edge_attr=[8107], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 6037], edge_attr=[6037], y=[199]),
  Data(x=[199, 12], edge_index=[2, 3975], edge_attr=[3975], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 8273], edge_attr=[8273], y=[199]),
  Data(x=[199, 12], edge_index=[2, 22589], edge_attr=[22589], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 8107], edge_attr=[8107], y=[199]),
  Data(x=[199, 12], edge_index=[2, 21298], edge_attr=[21298], y=[199]),
  tensor([1.])],
 [Data(x=[199, 12], edge_index=[2, 21298], edge_

In [12]:
input_dim = train_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, train_data, val_data)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:14<06:50, 14.14s/it]

Epoch: 1, Training Loss: 0.6475932522440405, Validation Loss: 0.6338973568600752, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


  7%|▋         | 2/30 [00:28<06:36, 14.15s/it]

Epoch: 2, Training Loss: 0.631888928863073, Validation Loss: 0.6296926105321076, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


 10%|█         | 3/30 [00:42<06:20, 14.08s/it]

...

Epoch: 28, Training Loss: 0.544024511875902, Validation Loss: 0.5266065711703828, Validation Accuracy: 0.7682672233820459, Validation F1 Score: 0.8393632416787264, Validation F2 Score: 0.8798543689320388, Validation F0.5 Score: 0.8024349750968456


 97%|█████████▋| 29/30 [06:58<00:14, 14.63s/it]

Epoch: 29, Training Loss: 0.5440046424620328, Validation Loss: 0.5263311030471499, Validation Accuracy: 0.7682672233820459, Validation F1 Score: 0.8393632416787264, Validation F2 Score: 0.8798543689320388, Validation F0.5 Score: 0.8024349750968456


100%|██████████| 30/30 [07:13<00:00, 14.45s/it]

Epoch: 30, Training Loss: 0.5466404375050883, Validation Loss: 0.5265062872888151, Validation Accuracy: 0.7682672233820459, Validation F1 Score: 0.8393632416787264, Validation F2 Score: 0.8798543689320388, Validation F0.5 Score: 0.8024349750968456


In [18]:
input_dim = train_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, train_data, val_data)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:11<05:34, 11.52s/it]

Epoch: 1, Training Loss: 0.6481233633735461, Validation Loss: 0.6358813381518601, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


  7%|▋         | 2/30 [00:21<04:54, 10.52s/it]

Epoch: 2, Training Loss: 0.6329547450833933, Validation Loss: 0.6256608172224558, Validation Accuracy: 0.6680584551148225, Validation F1 Score: 0.8005018820577164, Validation F2 Score: 0.9093500570125428, Validation F0.5 Score: 0.7149260421335724


 10%|█         | 3/30 [00:31<04:41, 10.44s/it]

...

Epoch: 28, Training Loss: 0.49227350000428305, Validation Loss: 0.48069631051668793, Validation Accuracy: 0.8100208768267223, Validation F1 Score: 0.8690647482014389, Validation F2 Score: 0.914043583535109, Validation F0.5 Score: 0.8283049917718047


 97%|█████████▋| 29/30 [04:51<00:10, 10.52s/it]

Epoch: 29, Training Loss: 0.4883902911388762, Validation Loss: 0.48038555467054095, Validation Accuracy: 0.8100208768267223, Validation F1 Score: 0.8690647482014389, Validation F2 Score: 0.914043583535109, Validation F0.5 Score: 0.8283049917718047


100%|██████████| 30/30 [05:00<00:00, 10.03s/it]

Epoch: 30, Training Loss: 0.49477145925958327, Validation Loss: 0.4803818877716403, Validation Accuracy: 0.8121085594989561, Validation F1 Score: 0.8706896551724138, Validation F2 Score: 0.9165154264972777, Validation F0.5 Score: 0.8292282430213465


In [19]:
input_dim = train_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, train_data, val_data)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:10<04:57, 10.25s/it]

Epoch: 1, Training Loss: 0.6480919939534916, Validation Loss: 0.6365035688951767, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


  7%|▋         | 2/30 [00:20<04:45, 10.20s/it]

Epoch: 2, Training Loss: 0.6448064789627538, Validation Loss: 0.6366730009464232, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


 10%|█         | 3/30 [00:31<04:42, 10.46s/it]

...

Epoch: 28, Training Loss: 0.6367510453785226, Validation Loss: 0.6343132530597655, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


 97%|█████████▋| 29/30 [05:05<00:11, 11.04s/it]

Epoch: 29, Training Loss: 0.6344919411802242, Validation Loss: 0.6343000412360611, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


100%|██████████| 30/30 [05:15<00:00, 10.52s/it]

Epoch: 30, Training Loss: 0.6359280158795783, Validation Loss: 0.6342895552982617, Validation Accuracy: 0.6659707724425887, Validation F1 Score: 0.7994987468671679, Validation F2 Score: 0.9088319088319088, Validation F0.5 Score: 0.7136465324384788


## sGNN with Feature Subset

In [20]:
with open("feature_dicts/filtered_features_dict.pkl", "rb") as f:
    feat_dict = pkl.load(f)

In [21]:
def add_features(years, graphs, feat_dict, dim):

    zeros = torch.zeros(dim)

    for i in range(len(years)):
        new_x = torch.empty(0, dim)
        year = years[i]

        feat_dict_year = feat_dict[year].combined_features

        for j, country in enumerate(all_nodes):
            if j == 0:
                new_x = torch.stack([zeros])

            elif country in feat_dict_year["country_code"].values:
                tensor_before = graphs[i].x[j]
                country_row = feat_dict_year[feat_dict_year["country_code"] == country]
                country_row = country_row.drop(columns = ["country_code", "current_gdp_growth"])
                row_values = country_row.values.tolist()
                row_tensor = torch.tensor(row_values)[0]
                combined_values = torch.cat((tensor_before, row_tensor))

                new_x = torch.cat((new_x, combined_values.unsqueeze(0)), dim=0)

            else:
                new_x = torch.cat((new_x, zeros.unsqueeze(0)), dim=0)

        graphs[i].x = new_x

    return graphs

In [22]:
all_graphs = add_features(years, all_graphs, feat_dict, 27)

In [23]:
labels = dist_labels_to_changepoint_labels(phases)
graph_pairs = sample_pairs(all_graphs,labels)

798 positive and 540 negative examples


In [24]:
graph_pairs_train = [graph_pairs[i] for i in train_indices]
graph_pairs_test = [graph_pairs[i] for i in test_indices]
graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [25]:
import random
positive_samples = [item for item in graph_pairs_train if item[2] == 1]
negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = graph_pairs_train

random.shuffle(balanced_data)

In [26]:
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:14<07:13, 14.95s/it]

Epoch: 1, Training Loss: 0.6528520211966683, Validation Loss: 0.6442973780482696, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7888748419721872, Validation F2 Score: 0.903300521134916, Validation F0.5 Score: 0.7001795332136446


  7%|▋         | 2/30 [00:28<06:40, 14.32s/it]

Epoch: 2, Training Loss: 0.6406836706206841, Validation Loss: 0.64019581999062, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7888748419721872, Validation F2 Score: 0.903300521134916, Validation F0.5 Score: 0.7001795332136446


 10%|█         | 3/30 [00:43<06:33, 14.58s/it]

...

Epoch: 28, Training Loss: 0.5169264792212622, Validation Loss: 0.5270805072684875, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8313253012048193, Validation F2 Score: 0.8625, Validation F0.5 Score: 0.8023255813953488


 97%|█████████▋| 29/30 [07:16<00:14, 14.59s/it]

Epoch: 29, Training Loss: 0.5086185338137167, Validation Loss: 0.5270868931782271, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8313253012048193, Validation F2 Score: 0.8625, Validation F0.5 Score: 0.8023255813953488


100%|██████████| 30/30 [07:31<00:00, 15.03s/it]

Epoch: 30, Training Loss: 0.5082922933648497, Validation Loss: 0.5269264378789074, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8313253012048193, Validation F2 Score: 0.8625, Validation F0.5 Score: 0.8023255813953488


In [35]:
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:10<05:12, 10.76s/it]

Epoch: 1, Training Loss: 0.6463643824418884, Validation Loss: 0.6326048406529278, Validation Accuracy: 0.6534446764091858, Validation F1 Score: 0.7882653061224489, Validation F2 Score: 0.8982558139534884, Validation F0.5 Score: 0.7022727272727273


  7%|▋         | 2/30 [00:20<04:43, 10.14s/it]

Epoch: 2, Training Loss: 0.6161229763035988, Validation Loss: 0.609263271528893, Validation Accuracy: 0.6534446764091858, Validation F1 Score: 0.7780748663101604, Validation F2 Score: 0.8640142517814727, Validation F0.5 Score: 0.7076848249027238


 10%|█         | 3/30 [00:31<04:48, 10.69s/it]

...

Epoch: 28, Training Loss: 0.376301123711009, Validation Loss: 0.38093539425376066, Validation Accuracy: 0.8705636743215032, Validation F1 Score: 0.9040247678018576, Validation F2 Score: 0.922882427307206, Validation F0.5 Score: 0.8859223300970874


 97%|█████████▋| 29/30 [04:49<00:09,  9.45s/it]

Epoch: 29, Training Loss: 0.37827936313119925, Validation Loss: 0.38092407350873647, Validation Accuracy: 0.8705636743215032, Validation F1 Score: 0.9040247678018576, Validation F2 Score: 0.922882427307206, Validation F0.5 Score: 0.8859223300970874


100%|██████████| 30/30 [04:58<00:00,  9.95s/it]

Epoch: 30, Training Loss: 0.3790532212116626, Validation Loss: 0.38078919964568353, Validation Accuracy: 0.8705636743215032, Validation F1 Score: 0.9040247678018576, Validation F2 Score: 0.922882427307206, Validation F0.5 Score: 0.8859223300970874


In [36]:
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:10<04:57, 10.28s/it]

Epoch: 1, Training Loss: 0.6547527795301336, Validation Loss: 0.6463807629020827, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7888748419721872, Validation F2 Score: 0.903300521134916, Validation F0.5 Score: 0.7001795332136446


  7%|▋         | 2/30 [00:20<04:42, 10.09s/it]

Epoch: 2, Training Loss: 0.648663713889685, Validation Loss: 0.6464087550600287, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7888748419721872, Validation F2 Score: 0.903300521134916, Validation F0.5 Score: 0.7001795332136446


...

Epoch: 28, Training Loss: 0.6397936031798459, Validation Loss: 0.6445685593967397, Validation Accuracy: 0.6555323590814196, Validation F1 Score: 0.7908745247148289, Validation F2 Score: 0.9043478260869565, Validation F0.5 Score: 0.7027027027027027


 97%|█████████▋| 29/30 [04:59<00:10, 10.17s/it]

Epoch: 29, Training Loss: 0.6403837327299446, Validation Loss: 0.6445352594389547, Validation Accuracy: 0.6555323590814196, Validation F1 Score: 0.7908745247148289, Validation F2 Score: 0.9043478260869565, Validation F0.5 Score: 0.7027027027027027


100%|██████████| 30/30 [05:09<00:00, 10.32s/it]

Epoch: 30, Training Loss: 0.638932671510812, Validation Loss: 0.6445111689587476, Validation Accuracy: 0.6555323590814196, Validation F1 Score: 0.7908745247148289, Validation F2 Score: 0.9043478260869565, Validation F0.5 Score: 0.7027027027027027


## No GDP 27 Features

## Random Feature Subset

In [19]:
years = range(1962,2019)

In [20]:
with open("feature_dicts/random_features_dict.pkl", "rb") as f:
    feat_dict_random = pkl.load(f)

In [21]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [22]:
all_graphs = add_features(years, all_graphs, feat_dict_random, 27)

In [23]:
labels = dist_labels_to_changepoint_labels(phases)
graph_pairs = sample_pairs(all_graphs,labels)

798 positive and 540 negative examples


In [24]:
graph_pairs_train = [graph_pairs[i] for i in train_indices]
graph_pairs_test = [graph_pairs[i] for i in test_indices]
graph_pairs_val = [graph_pairs[i] for i in val_indices]

In [25]:
import random
positive_samples = [item for item in graph_pairs_train if item[2] == 1]
negative_samples = [item for item in graph_pairs_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = graph_pairs_train

random.shuffle(balanced_data)

In [24]:
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:14<07:11, 14.87s/it]

Epoch: 1, Training Loss: 0.6523301030899414, Validation Loss: 0.6295803544541242, Validation Accuracy: 0.6492693110647182, Validation F1 Score: 0.7862595419847328, Validation F2 Score: 0.8956521739130435, Validation F0.5 Score: 0.7006802721088435


  7%|▋         | 2/30 [00:29<06:48, 14.59s/it]

Epoch: 2, Training Loss: 0.6354912189356088, Validation Loss: 0.6180587373596144, Validation Accuracy: 0.6951983298538622, Validation F1 Score: 0.8027027027027027, Validation F2 Score: 0.8844550327575939, Validation F0.5 Score: 0.7347847600197922


 10%|█         | 3/30 [00:41<06:07, 13.63s/it]

...

Epoch: 28, Training Loss: 0.5452263315394131, Validation Loss: 0.5299973061662129, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8318318318318318, Validation F2 Score: 0.8629283489096573, Validation F0.5 Score: 0.8028985507246377


 97%|█████████▋| 29/30 [06:27<00:13, 13.58s/it]

Epoch: 29, Training Loss: 0.539252459489066, Validation Loss: 0.5297640619297864, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8318318318318318, Validation F2 Score: 0.8629283489096573, Validation F0.5 Score: 0.8028985507246377


100%|██████████| 30/30 [06:40<00:00, 13.34s/it]

Epoch: 30, Training Loss: 0.5405990651875828, Validation Loss: 0.5295222561518683, Validation Accuracy: 0.7661795407098121, Validation F1 Score: 0.8318318318318318, Validation F2 Score: 0.8629283489096573, Validation F0.5 Score: 0.8028985507246377


In [25]:
import itertools
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GraphSAGE(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:12<05:56, 12.30s/it]

Epoch: 1, Training Loss: 0.6483550843711682, Validation Loss: 0.6311088434589681, Validation Accuracy: 0.6430062630480167, Validation F1 Score: 0.7827191867852605, Validation F2 Score: 0.9000584453535944, Validation F0.5 Score: 0.6924460431654677


  7%|▋         | 2/30 [00:22<05:15, 11.26s/it]

Epoch: 2, Training Loss: 0.6282569432520194, Validation Loss: 0.6065233254606888, Validation Accuracy: 0.6951983298538622, Validation F1 Score: 0.7932011331444759, Validation F2 Score: 0.8588957055214724, Validation F0.5 Score: 0.7368421052631579


 10%|█         | 3/30 [00:35<05:20, 11.86s/it]

...

Epoch: 28, Training Loss: 0.44988762690854045, Validation Loss: 0.4228902540027722, Validation Accuracy: 0.8496868475991649, Validation F1 Score: 0.8899082568807339, Validation F2 Score: 0.9220532319391636, Validation F0.5 Score: 0.8599290780141844


 97%|█████████▋| 29/30 [05:13<00:10, 10.09s/it]

Epoch: 29, Training Loss: 0.44842066361238964, Validation Loss: 0.42277375548020285, Validation Accuracy: 0.8496868475991649, Validation F1 Score: 0.8899082568807339, Validation F2 Score: 0.9220532319391636, Validation F0.5 Score: 0.8599290780141844


100%|██████████| 30/30 [05:23<00:00, 10.80s/it]

Epoch: 30, Training Loss: 0.45029023526355144, Validation Loss: 0.42269675256315004, Validation Accuracy: 0.8475991649269311, Validation F1 Score: 0.888208269525268, Validation F2 Score: 0.9194673430564363, Validation F0.5 Score: 0.8590047393364929


In [26]:
import itertools
input_dim = balanced_data[0][0].x.shape[1]

val_losses = []
parameters = []
# Define hyperparameter grids
learning_rates = [0.01]
dropout_rates = [0.05]
sort_k_values = [50]
hidden_units_values = [16]

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(learning_rates, dropout_rates, sort_k_values, hidden_units_values))

for lr, dropout_rate, sort_k, hidden_units in hyperparameter_combinations:
    model = SiameseGNN_GIN(sort_k, input_dim, dropout = dropout_rate, nhidden=hidden_units)
    val_accuracy, val_f1, val_loss, val_f2, val_05 = run_model(model, balanced_data, graph_pairs_val)

    parameters.append((lr, dropout_rate, sort_k, hidden_units))
    val_losses.append((val_accuracy, val_f1, val_loss, val_f2, val_05))

  3%|▎         | 1/30 [00:11<05:19, 11.00s/it]

Epoch: 1, Training Loss: 0.6485064943382359, Validation Loss: 0.6554418763686322, Validation Accuracy: 0.6409185803757829, Validation F1 Score: 0.7811704834605598, Validation F2 Score: 0.8992384299941417, Validation F0.5 Score: 0.6905083220872694


  7%|▋         | 2/30 [00:21<04:58, 10.66s/it]

Epoch: 2, Training Loss: 0.6393981634511718, Validation Loss: 0.6548049792740689, Validation Accuracy: 0.6409185803757829, Validation F1 Score: 0.7811704834605598, Validation F2 Score: 0.8992384299941417, Validation F0.5 Score: 0.6905083220872694


 10%|█         | 3/30 [00:31<04:38, 10.33s/it]

...

Epoch: 28, Training Loss: 0.6327292657259739, Validation Loss: 0.6467693295633121, Validation Accuracy: 0.6492693110647182, Validation F1 Score: 0.7823834196891192, Validation F2 Score: 0.8919078558771412, Validation F0.5 Score: 0.6968158744808491


 97%|█████████▋| 29/30 [05:03<00:10, 10.59s/it]

Epoch: 29, Training Loss: 0.6329341389106483, Validation Loss: 0.6467284303866249, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7833981841763943, Validation F2 Score: 0.892434988179669, Validation F0.5 Score: 0.6981044845122515


100%|██████████| 30/30 [05:13<00:00, 10.46s/it]

Epoch: 30, Training Loss: 0.6285209690328676, Validation Loss: 0.6466965342488816, Validation Accuracy: 0.651356993736952, Validation F1 Score: 0.7833981841763943, Validation F2 Score: 0.892434988179669, Validation F0.5 Score: 0.6981044845122515
